# Imports

In [ ]:
import os
import glob
import warnings
import pyarrow.parquet as pq

from IPython.display import display

import numpy as np
import pandas as pd

import getpass
import os

user = getpass.getuser()

# Configuration

In [ ]:
# =========================================================
# CONFIGURATION SECTION
# =========================================================
#input_paths = [
#    f"/scratch/users/{user}/speczs-catalogs/processed/2dfgrs_final_release.parquet",
#    f"/scratch/users/{user}/speczs-catalogs/processed/2dflens_final_release.parquet",
#    f"/scratch/users/{user}/speczs-catalogs/processed/2mrs_v240.parquet",
#    f"/scratch/users/luigi.silva/speczs-catalogs/processed/3dhst_v4.1.5.parquet",
#    f"/scratch/users/{user}/speczs-catalogs/processed/6dfgs_dr3.parquet",
#    f"/scratch/users/luigi.silva/speczs-catalogs/processed/astrodeep_jwst.parquet",
#    f"/scratch/users/luigi.silva/speczs-catalogs/processed/astrodeep-gs43.parquet",
#    f"/scratch/users/{user}/speczs-catalogs/processed/desi_dr1_in_lsst_dp1_fields.parquet",
#    f"/scratch/users/{user}/speczs-catalogs/processed/jades_dr3.parquet",
#    f"/scratch/users/{user}/speczs-catalogs/processed/mosdef_final_release.parquet",
#    f"/scratch/users/{user}/speczs-catalogs/processed/ozdes_dr2.parquet",
#    f"/scratch/users/{user}/speczs-catalogs/processed/primus_dr1.parquet",
#    f"/scratch/users/{user}/speczs-catalogs/processed/vandels_dr4.parquet",
#    f"/scratch/users/luigi.silva/speczs-catalogs/processed/vlt_vimos_v2.0.1.parquet",
#    f"/scratch/users/luigi.silva/speczs-catalogs/processed/vuds_dr1.parquet",
#    f"/scratch/users/{user}/speczs-catalogs/processed/vvds_final_release.parquet",
#    f"/scratch/users/luigi.silva/speczs-catalogs/johns-catalogs/z_cat_CANDELS_clean_sitcomtn-154.parquet",
#    f"/scratch/users/luigi.silva/speczs-catalogs/johns-catalogs/z_cat_NED_clean_sitcomtn-154.parquet",
#]

input_paths = glob.glob('test_data/*.parquet')

# replace paths in case of using local env 
final_catalog_path = f"./process001/outputs/crd.parquet"
prepared_temp_dir = f"./process001/temp/"

combine_mode = "concatenate_and_mark_duplicates" # Options: "concatenate", "concatenate_and_mark_duplicates", or "concatenate_and_remove_duplicates"

# Validation

## Basic Info

Counting input and output rows.

In [ ]:
# ========================================================
# COUNT INPUT ROWS
# =========================================================
total_input_rows = 0
for path in input_paths:
    if os.path.exists(path):
        parquet_file = pq.ParquetFile(path)
        n_rows = parquet_file.metadata.num_rows
        print(f"{path} -> {n_rows} rows")
        total_input_rows += n_rows
    else:
        warnings.warn(f"⚠️ File not found: {path}")

print(f"✅ Total number of input rows: {total_input_rows}")

# =========================================================
# LOAD FINAL MERGED CATALOG
# =========================================================
if not os.path.exists(final_catalog_path):
    raise FileNotFoundError(f"❌ Final catalog not found: {final_catalog_path}")

df_final = pd.read_parquet(final_catalog_path)
print(f"✅ Total number of rows in final catalog: {len(df_final)}")

In [ ]:
df_final

In [ ]:
df_final.columns

In [ ]:
df_final.dtypes

Basic statistics.

In [ ]:
df_final.describe()

Counting tie_result values.

In [ ]:
if combine_mode != "concatenate":
    print(df_final["tie_result"].value_counts())

Counting survey values.

In [ ]:
df_final["source"].value_counts()

Checking the percentage of unsolved objects.

In [ ]:
if combine_mode != "concatenate":
    # Total number of objects
    total_all = len(df_final)
    
    # Filter objects that were compared (compared_to is not null or empty)
    mask_compared = df_final["compared_to"].notna() & (df_final["compared_to"] != "")
    df_compared = df_final[mask_compared]
    
    # Count how many have tie_result == 2
    count_tie2 = (df_final["tie_result"] == 2).sum()
    count_tie2_compared = (df_compared["tie_result"] == 2).sum()
    
    # Percentages
    percent_all = (count_tie2 / total_all) * 100 if total_all > 0 else 0
    percent_compared = (count_tie2_compared / len(df_compared)) * 100 if len(df_compared) > 0 else 0
    
    # Formatted print
    print(f"📊 tie_result == 2 represents:")
    print(f"  • {percent_all:.2f}% of the total ({count_tie2} out of {total_all})")
    print(f"  • {percent_compared:.2f}% of the compared objects ({count_tie2_compared} out of {len(df_compared)})")

## Automatic Validation

All rules.

In [ ]:
# validator_crc.py
from __future__ import annotations
import math
from collections import defaultdict

import numpy as np
import pandas as pd


# ------------------------------
# Utilities & normalization
# ------------------------------

TYPE_PRIORITY = {"s": 3, "g": 2, "p": 1}  # others/NA -> 0


def _norm_str(s):
    """Normalize string-like fields; returns None if empty-like."""
    if s is None:
        return None
    s = str(s).strip()
    if not s or s.lower() in {"na", "nan", "<na>", "none", "null"}:
        return None
    return s


def _type_score(t):
    """Map instrument_type_homogenized to numeric priority."""
    t = _norm_str(t)
    return TYPE_PRIORITY.get(t, 0)


def _to_num(x):
    """Best-effort numeric cast -> float; returns np.nan on failure."""
    try:
        return float(x)
    except Exception:
        return np.nan


def _z_diff(a, b):
    """Absolute z difference; returns np.inf if either is NA (undefined)."""
    if a is None or b is None:
        return np.inf
    if (isinstance(a, float) and math.isnan(a)) or (isinstance(b, float) and math.isnan(b)):
        return np.inf
    return abs(a - b)


def _is_na(x):
    return x is None or (isinstance(x, float) and math.isnan(x))


# ------------------------------
# Graph building from compared_to  (EXCLUDES STARS)
# ------------------------------

def build_components(df: pd.DataFrame, excluded_ids: set[str] | None = None) -> list[tuple[str, ...]]:
    """
    Build undirected components using 'CRD_ID' and 'compared_to' comma-list.

    - Only rows with non-empty compared_to contribute edges.
    - Any edge touching IDs in `excluded_ids` (e.g., stars) is dropped.
    - Nodes in `excluded_ids` are not included in the components.

    Args:
        df: DataFrame with columns CRD_ID, compared_to.
        excluded_ids: Optional set of IDs to exclude from edges/nodes.

    Returns:
        List of sorted tuples, one per connected component.
    """
    excluded_ids = excluded_ids or set()

    edges = []
    seen_nodes = set()

    for _, row in df.iterrows():
        crd = _norm_str(row.get("CRD_ID"))
        if not crd or crd in excluded_ids:
            continue
        cmp_raw = _norm_str(row.get("compared_to"))
        if not cmp_raw:
            continue

        for nb in cmp_raw.split(","):
            nb = _norm_str(nb)
            if not nb:
                continue
            if nb in excluded_ids:
                # drop edges touching excluded nodes
                continue
            edges.append((crd, nb))
            seen_nodes.add(crd)
            seen_nodes.add(nb)

    # Disjoint-set (Union-Find)
    parent = {}

    def find(x):
        parent.setdefault(x, x)
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]

    def union(a, b):
        ra, rb = find(a), find(b)
        if ra != rb:
            parent[rb] = ra

    for a, b in edges:
        union(a, b)

    for n in seen_nodes:
        find(n)

    comps = defaultdict(list)
    for n in parent.keys():
        comps[find(n)].append(n)

    return [tuple(sorted(v)) for v in comps.values()]


# ------------------------------
# Pair validation rules  (GUARDS AGAINST STARS)
# ------------------------------

def validate_pair(group_df: pd.DataFrame, threshold: float) -> list[dict]:
    """
    Validate a 2-row group against the pair rules.
    Returns a list of violation dicts (empty if valid).

    New behavior:
      - If any member is a star (z_flag_homogenized==6) OR tie_result==3,
        this pair should not exist (stars are excluded from comparisons).
        A violation 'PAIR_STAR_SHOULD_NOT_BE_COMPARED' is emitted and other
        checks for this pair are skipped.
      - Stars must carry tie_result==3; otherwise 'STAR_MUST_BE_3'.
    """
    vios = []

    a, b = group_df.iloc[0], group_df.iloc[1]

    tr_a, tr_b = int(_to_num(a["tie_result"])), int(_to_num(b["tie_result"]))
    zf_a = _to_num(a["z_flag_homogenized"])
    zf_b = _to_num(b["z_flag_homogenized"])

    # Star checks
    a_is_star = (zf_a == 6) or (tr_a == 3)
    b_is_star = (zf_b == 6) or (tr_b == 3)

    if a_is_star and tr_a != 3:
        vios.append({
            "rule": "STAR_MUST_BE_3",
            "message": "Row with z_flag_homogenized==6 must have tie_result==3.",
            "group_ids": tuple(group_df["CRD_ID"].tolist()),
            "rows": group_df.copy(),
        })
    if b_is_star and tr_b != 3:
        vios.append({
            "rule": "STAR_MUST_BE_3",
            "message": "Row with z_flag_homogenized==6 must have tie_result==3.",
            "group_ids": tuple(group_df["CRD_ID"].tolist()),
            "rows": group_df.copy(),
        })

    if a_is_star or b_is_star:
        vios.append({
            "rule": "PAIR_STAR_SHOULD_NOT_BE_COMPARED",
            "message": "Stars must be excluded from comparisons; this pair should not exist.",
            "group_ids": tuple(group_df["CRD_ID"].tolist()),
            "rows": group_df.copy(),
        })
        return vios  # skip further pair logic

    # --- original pair logic (unchanged) ---
    # For comparisons, ignore flag 6 as "quality"
    zf_a_eff = (-1 if zf_a == 6 else zf_a)
    zf_b_eff = (-1 if zf_b == 6 else zf_b)

    ts_a = _type_score(a["instrument_type_homogenized"])
    ts_b = _type_score(b["instrument_type_homogenized"])

    z_a = _to_num(a["z"])
    z_b = _to_num(b["z"])
    dz = _z_diff(z_a, z_b)  # np.inf if undefined

    # Case: (1,0) or (0,1)
    if {tr_a, tr_b} == {0, 1}:
        win = a if tr_a == 1 else b
        los = b if tr_a == 1 else a

        win_zf = _to_num(win["z_flag_homogenized"])
        los_zf = _to_num(los["z_flag_homogenized"])
        win_zf_eff = (-1 if win_zf == 6 else win_zf)
        los_zf_eff = (-1 if los_zf == 6 else los_zf)

        win_ts = _type_score(win["instrument_type_homogenized"])
        los_ts = _type_score(los["instrument_type_homogenized"])

        win_z = _to_num(win["z"])
        los_z = _to_num(los["z"])
        win_los_dz = _z_diff(win_z, los_z)

        cond = (
            (win_zf_eff > los_zf_eff) or
            (win_zf_eff == los_zf_eff and win_ts > los_ts) or
            (win_zf_eff == los_zf_eff and win_ts == los_ts and win_los_dz < threshold)
        )
        if not cond:
            vios.append({
                "rule": "PAIR_1v0_PRIORITY",
                "message": "Winner does not have higher z_flag (excl. 6), nor better type, nor Δz < threshold on tie.",
                "group_ids": tuple(group_df["CRD_ID"].tolist()),
                "rows": group_df.copy(),
            })

    # Case: (2,2)
    elif tr_a == 2 and tr_b == 2:
        cond_equal_quality = (zf_a_eff == zf_b_eff) and (ts_a == ts_b)
        cond_delta = (dz > threshold) or (math.isinf(dz))
        if not (cond_equal_quality and cond_delta):
            vios.append({
                "rule": "PAIR_2v2_TIE_CONSISTENCY",
                "message": "Tie (2,2) requires equal z_flag (excl. 6), equal type, and Δz > threshold (or undefined).",
                "group_ids": tuple(group_df["CRD_ID"].tolist()),
                "rows": group_df.copy(),
            })

    # Case: (0,0)
    elif tr_a == 0 and tr_b == 0:
        # With star exclusion, (0,0) should be rare; keep original check
        if not (zf_a == 6 and zf_b == 6):
            vios.append({
                "rule": "PAIR_0v0_BOTH_FLAG6",
                "message": "Both eliminated in a pair must have z_flag_homogenized == 6.",
                "group_ids": tuple(group_df["CRD_ID"].tolist()),
                "rows": group_df.copy(),
            })

    else:
        vios.append({
            "rule": "PAIR_INVALID_TIE_PATTERN",
            "message": f"Unexpected tie_result pattern for pair: ({tr_a},{tr_b}).",
            "group_ids": tuple(group_df["CRD_ID"].tolist()),
            "rows": group_df.copy(),
        })

    return vios


# ------------------------------
# Group validation rules (size >= 3)  (GUARDS AGAINST STARS)
# ------------------------------

def validate_group(group_df: pd.DataFrame, threshold: float) -> list[dict]:
    """
    Validate a group (>=3) with hierarchical constraints.

    New behavior:
      - If any member is a star (z_flag_homogenized==6 OR tie_result==3),
        this group should not exist (stars are excluded from comparisons).
        Emit 'GROUP_STAR_SHOULD_NOT_BE_COMPARED' and skip the rest.
      - Stars must carry tie_result==3; otherwise 'STAR_MUST_BE_3'.
    """
    vios = []
    df = group_df.copy()

    # Early star checks
    zf = pd.to_numeric(df["z_flag_homogenized"], errors="coerce")
    tr = pd.to_numeric(df["tie_result"], errors="coerce").fillna(-1).astype(int)
    star_mask = (zf == 6) | (tr == 3)
    if star_mask.any():
        # Any star must have tie_result==3
        bad_star = star_mask & (tr != 3)
        if bad_star.any():
            vios.append({
                "rule": "STAR_MUST_BE_3",
                "message": "Rows with z_flag_homogenized==6 must have tie_result==3.",
                "group_ids": tuple(df["CRD_ID"].tolist()),
                "rows": df.copy(),
            })
        vios.append({
            "rule": "GROUP_STAR_SHOULD_NOT_BE_COMPARED",
            "message": "Stars must be excluded from comparisons; this group should not exist.",
            "group_ids": tuple(df["CRD_ID"].tolist()),
            "rows": df.copy(),
        })
        return vios  # skip further group logic

    # --- original group logic (unchanged beyond star guard) ---
    it = df["instrument_type_homogenized"].map(_type_score).astype("Int64")
    z = pd.to_numeric(df["z"], errors="coerce")

    # Survivors = rows with tie_result in {1,2}
    survivors_mask = tr.isin({1, 2})
    survivors_idx = df.index[survivors_mask]
    losers_idx = df.index[~survivors_mask]

    # 0) All-flag-6 special case: all must be 0 (shouldn't happen now)
    if zf.notna().all() and (zf == 6).all():
        if survivors_mask.any():
            vios.append({
                "rule": "GROUP_ALL_FLAG6_ALL_ZERO",
                "message": "All members have flag 6 but there are survivors.",
                "group_ids": tuple(df["CRD_ID"].tolist()),
                "rows": df.copy(),
            })
        return vios

    # 1) Flag dominance (ignore 6 as quality)
    zf_eff = zf.where(zf != 6, other=-1)
    max_flag = zf_eff.max(skipna=True)
    bad_flag_survivors = survivors_idx[zf_eff.loc[survivors_idx] < max_flag]
    if len(bad_flag_survivors) > 0:
        vios.append({
            "rule": "GROUP_FLAG_DOMINANCE",
            "message": "Survivors include members with lower z_flag than group max (excluding 6).",
            "group_ids": tuple(df["CRD_ID"].tolist()),
            "rows": df.copy(),
        })

    # 2) Type dominance among max-flag candidates
    max_flag_mask = (zf_eff == max_flag)
    if max_flag_mask.any():
        max_type = it[max_flag_mask].max(skipna=True)
        bad_type_survivors = survivors_idx[it.loc[survivors_idx] < max_type]
        if len(bad_type_survivors) > 0:
            vios.append({
                "rule": "GROUP_TYPE_DOMINANCE",
                "message": "Survivors include members with lower instrument_type than group max.",
                "group_ids": tuple(df["CRD_ID"].tolist()),
                "rows": df.copy(),
            })

    # 3) Δz-threshold independence among survivors at max-flag & max-type
    cand_mask = (zf_eff == max_flag) & (it == it[max_flag_mask].max(skipna=True))
    cand_idx = df.index[cand_mask]
    crd = df.loc[cand_idx, "CRD_ID"].astype(str).tolist()
    zvals = z.loc[cand_idx].to_numpy()

    survivors_crd = set(df.loc[survivors_idx, "CRD_ID"].astype(str))

    for i in range(len(crd)):
        for j in range(i + 1, len(crd)):
            zi, zj = zvals[i], zvals[j]
            if not (np.isnan(zi) or np.isnan(zj)):
                if abs(zi - zj) < threshold and (crd[i] in survivors_crd) and (crd[j] in survivors_crd):
                    vios.append({
                        "rule": "GROUP_DELTZ_INDEPENDENCE",
                        "message": "Two survivors are closer than threshold (both z defined). Survivors must form an independent set.",
                        "group_ids": tuple(df["CRD_ID"].tolist()),
                        "rows": df.copy(),
                    })
                    break

    # 4) Final labeling consistency
    n_survivors = int(survivors_mask.sum())
    if n_survivors == 0:
        if not (zf.notna().all() and (zf == 6).all()):
            vios.append({
                "rule": "GROUP_NO_SURVIVOR_SUSPECT",
                "message": "No survivors but group not all flag 6.",
                "group_ids": tuple(df["CRD_ID"].tolist()),
                "rows": df.copy(),
            })
    elif n_survivors == 1:
        lone_idx = survivors_idx[0]
        if int(df.loc[lone_idx, "tie_result"]) != 1:
            vios.append({
                "rule": "GROUP_SINGLE_SURVIVOR_MUST_BE_1",
                "message": "Exactly one survivor but not labeled with tie_result == 1.",
                "group_ids": tuple(df["CRD_ID"].tolist()),
                "rows": df.copy(),
            })
    else:
        if not (df.loc[survivors_idx, "tie_result"] == 2).all():
            vios.append({
                "rule": "GROUP_MULTI_SURVIVOR_MUST_BE_2",
                "message": "Multiple survivors but not all labeled tie_result == 2.",
                "group_ids": tuple(df["CRD_ID"].tolist()),
                "rows": df.copy(),
            })

    return vios


# ------------------------------
# Public API  (excludes stars from graph + enforces tie_result==3)
# ------------------------------

def validate_tie_results(
    df_final: pd.DataFrame,
    threshold: float = 0.0005,
    max_groups: int | None = None,
) -> dict:
    """
    Main entry point.
    - Excludes stars (z_flag_homogenized==6 or tie_result==3) from graph.
    - Builds components from 'compared_to' among NON-stars.
    - Validates pairs and groups with the declared rules.
    - Validates that all stars have tie_result==3.
    """
    df = df_final.copy()
    df["CRD_ID"] = df["CRD_ID"].astype(str)
    df["compared_to"] = df["compared_to"].astype("string")

    # Identify stars & enforce tie_result==3
    zf = pd.to_numeric(df.get("z_flag_homogenized"), errors="coerce")
    tr = pd.to_numeric(df.get("tie_result"), errors="coerce")
    star_mask = (zf == 6) | (tr == 3)
    star_ids = set(df.loc[star_mask, "CRD_ID"].astype(str))
    # Pre-collect violations for stars with wrong tie_result
    violations = []
    wrong_tr_mask = (zf == 6) & (tr != 3)
    if wrong_tr_mask.any():
        rows = df.loc[wrong_tr_mask].copy()
        violations.append({
            "rule": "STAR_MUST_BE_3",
            "message": "Rows with z_flag_homogenized==6 must have tie_result==3.",
            "group_ids": tuple(rows["CRD_ID"].tolist()),
            "rows": rows,
        })

    # Build components EXCLUDING stars
    components = build_components(df, excluded_ids=star_ids)
    if max_groups is not None:
        components = components[:max_groups]

    n_pairs = n_groups = 0

    for comp in components:
        group_df = df[df["CRD_ID"].isin(comp)].copy()
        if len(group_df) == 2:
            n_pairs += 1
            violations.extend(validate_pair(group_df, threshold))
        elif len(group_df) > 2:
            n_groups += 1
            violations.extend(validate_group(group_df, threshold))
        # singleton components are irrelevant here

    summary = {
        "n_components": len(components),
        "n_pairs": n_pairs,
        "n_groups": n_groups,
        "n_violations": len(violations),
        "by_rule": pd.Series([v["rule"] for v in violations]).value_counts().to_dict() if violations else {},
        "n_stars_excluded": int(star_mask.sum()),
    }
    return {"summary": summary, "violations": violations}

In [ ]:
report = validate_tie_results(df_final, threshold=0.0005, max_groups=10000)

In [ ]:
print(report["summary"])

In [ ]:
# Inspecione alguns casos
viol = report["violations"]
for v in viol[:5]:
    print(v["rule"], "->", v["message"], "IDs:", v["group_ids"])
    display(v["rows"][[
        "CRD_ID", "ra", "dec", "z","z_flag_homogenized","instrument_type_homogenized","tie_result","survey","source","compared_to"
    ]])

Checking if all values with compared_to <NA> have tie_result 1 (or 0, if z_flag_homogenized = 6)

In [ ]:
# --- Validation: when compared_to is <NA>, tie_result must be 1
# --- EXCEPTION: tie_result may be 3 ONLY if z_flag_homogenized == 6

import pandas as pd

# Ensure compared_to is normalized to real NA (StringDtype shows <NA>)
df_final_val = df_final.copy()
df_final_val["compared_to"] = (
    df_final_val["compared_to"]
      .astype("string").str.strip()
      .replace({"": pd.NA, "nan": pd.NA, "NaN": pd.NA, "NA": pd.NA,
                "<NA>": pd.NA, "None": pd.NA, "null": pd.NA})
)

# 1) Subset to rows where compared_to is truly missing
na_cmp = df_final_val[df_final_val["compared_to"].isna()].copy()

# 2) Parse tie_result and z_flag_homogenized (para lógica, mantendo -1)
tie_as_int   = pd.to_numeric(na_cmp["tie_result"], errors="coerce").fillna(-1).astype(int)
zflag_as_int = pd.to_numeric(na_cmp["z_flag_homogenized"], errors="coerce").fillna(-1).astype(int)

# 3) Valid if:
#    - tie_result == 1
#    - OR tie_result == 3 and z_flag_homogenized == 6
valid_mask = (tie_as_int.eq(1)) | (tie_as_int.eq(3) & zflag_as_int.eq(6))
violations = na_cmp[~valid_mask].copy()

# 4) Summary
total_na      = len(na_cmp)
valid_count   = int(valid_mask.sum())
invalid_count = len(violations)

print("===== compared_to <NA> validation (tie_result rule with flag-6 exception) =====")
print(f"Rows with compared_to <NA>: {total_na}")
print(f"Valid:                      {valid_count}")
print(f"INVALID:                    {invalid_count}")

# 4b) Exibição sem -1: converte para Int64 -> string -> substitui NA
tie_disp   = pd.to_numeric(na_cmp["tie_result"], errors="coerce").astype("Int64").astype("string").fillna("<NA>")
zflag_disp = pd.to_numeric(na_cmp["z_flag_homogenized"], errors="coerce").astype("Int64").astype("string").fillna("<NA>")

print("\nCrosstab of tie_result x z_flag_homogenized (for display, <NA> shown):")
print(pd.crosstab(tie_disp, zflag_disp, dropna=False))

# 5) Show some offending rows (if any)
if invalid_count > 0:
    cols_to_show = [
        "CRD_ID", "ra", "dec", "z", "z_flag", "z_err",
        "z_flag_homogenized", "instrument_type", "instrument_type_homogenized",
        "tie_result", "survey", "source", "compared_to"
    ]
    print("\n⚠️ Examples of violations (up to 10):")
    display(violations[cols_to_show].head(10))
else:
    print("\n✅ All rows with compared_to <NA> satisfy the rule (1, or 3 with flag==6).")

# 6) Optional hard assertion
# assert invalid_count == 0, "Found rows where compared_to is <NA> but tie_result is invalid (not 1, nor 3 with flag==6)."

In [ ]:
# Seleciona as linhas com z_flag_homogenized nulo (<NA>)
na_zflag = df_final[df_final["z_flag_homogenized"].isna()].copy()

print(f"Total de linhas com z_flag_homogenized <NA>: {len(na_zflag)}")

cols_to_show = [
    "CRD_ID", "ra", "dec", "z", "z_flag", "z_err",
    "z_flag_homogenized", "instrument_type", "instrument_type_homogenized",
    "tie_result", "survey", "source", "compared_to"
]

# Mostra algumas linhas
display(na_zflag[cols_to_show].head(20))

Checking if some z_flag_homogenized 6 entered the comparisson.

In [ ]:
import pandas as pd

# --- Normalize compared_to so blanks/strings like "nan" become NA ---
df_chk = df_final.copy()
df_chk["compared_to_norm"] = (
    df_chk["compared_to"]
      .astype("string").str.strip()
      .replace({"": pd.NA, "nan": pd.NA, "NaN": pd.NA, "NA": pd.NA,
                "<NA>": pd.NA, "None": pd.NA, "null": pd.NA})
)

# --- Check condition only on rows with z_flag_homogenized == 6 ---
mask6 = df_chk["z_flag_homogenized"].eq(6)
all_na = df_chk.loc[mask6, "compared_to_norm"].isna().all()

print(f"Todos os z_flag_homogenized==6 têm compared_to <NA>? {all_na}")

# Optional: list offending rows if any
if not all_na:
    offenders = df_chk.loc[
        mask6 & df_chk["compared_to_norm"].notna(),
        ["CRD_ID", "z", "z_flag_homogenized", "survey", "source", "compared_to", "tie_result"]
    ].head(20)
    display(offenders)

## Manual Validation

In [ ]:
do_manual_comparisson = False

if combine_mode != "concatenate" and do_manual_comparisson:
    # =========================================================
    # ANALYZE GROUPS BY COMPARED_TO
    # =========================================================
    import numpy as np
    from collections import defaultdict, deque

    threshold = 0.0005
    max_groups = 10000  # For debugging or limiting processed groups

    desired_order = [
        "CRD_ID", "id", "ra", "dec", "z", "z_flag", "z_err", "instrument_type", "survey", "source",
        "z_flag_homogenized", "instrument_type_homogenized", "tie_result", "compared_to", "role"
    ]

    # Filter valid rows: only rows with non-empty 'compared_to'
    df_final_just_compared = df_final[
        (df_final["compared_to"].notnull()) &
        (df_final["compared_to"] != "")
    ]
    print(f"✅ Number of rows with non-empty compared_to: {len(df_final_just_compared)}")

    # ------------------------------------------------------------------
    # Build an undirected adjacency graph from 'compared_to' relationships
    # ------------------------------------------------------------------
    adjacency = defaultdict(set)
    for _, row in df_final_just_compared.iterrows():
        crd_id = row["CRD_ID"]
        for neighbor in str(row["compared_to"]).split(","):
            nb = neighbor.strip()
            if not nb:
                continue
            adjacency[crd_id].add(nb)
            adjacency[nb].add(crd_id)

    # BFS to get connected component starting at 'start_id'
    def get_connected_group(start_id):
        visited = set()
        queue = deque([start_id])
        while queue:
            current = queue.popleft()
            if current in visited:
                continue
            visited.add(current)
            queue.extend(adjacency[current] - visited)
        return tuple(sorted(visited))

    # Buckets for case studies / inspections
    group_cases = {
        "CASE1_small_same": [],
        "CASE1_small_diff": [],
        "CASE1_large_same": [],
        "CASE1_large_diff": [],
        "CASE2_small_same": [],
        "CASE2_small_diff": [],
        "CASE2_large_same": [],
        "CASE2_large_diff": [],
        "TIE_FLAG_TYPE_BREAK_PAIR": [],
        "TIE_FLAG_TYPE_BREAK_GROUP": [],
        "SAME_FLAG_DIFF_TYPE": [],
        "SAME_SOURCE_PAIR": []  # NEW: pairs with identical `source`
    }

    processed_groups = 0
    seen_groups = set()

    # ------------------------------------------------------------------
    # Loop over rows; for each connected group, compute diagnostics once
    # ------------------------------------------------------------------
    for idx, row in df_final_just_compared.iterrows():
        if max_groups is not None and processed_groups >= max_groups:
            break

        group_ids = get_connected_group(row["CRD_ID"])
        if group_ids in seen_groups:
            continue
        seen_groups.add(group_ids)

        group_df = df_final_just_compared[df_final_just_compared["CRD_ID"].isin(group_ids)].copy()
        if len(group_df) < 2:
            continue

        # Mark the row used to discover the group (only for display)
        group_df["role"] = np.where(group_df["CRD_ID"] == row["CRD_ID"], "principal", "compared")

        # Pairwise Δz and survey signature
        z_vals = group_df["z"].to_numpy()
        surveys = group_df["survey"].values

        delta_z_matrix = np.abs(z_vals[:, None] - z_vals[None, :])
        pairwise_dz = delta_z_matrix[np.triu_indices(len(z_vals), k=1)]

        max_delta_z = float(np.max(pairwise_dz)) if len(pairwise_dz) else 0.0
        all_pairs_below_thresh = bool(np.all(pairwise_dz <= threshold)) if len(pairwise_dz) else True
        same_survey = len(set(surveys)) == 1

        # Classification by size (pair/group), Δz, and survey consistency
        if len(group_df) == 2:
            key = (
                "CASE1_small_same" if (max_delta_z <= threshold and same_survey) else
                "CASE1_small_diff" if (max_delta_z <= threshold) else
                "CASE1_large_same" if (same_survey) else
                "CASE1_large_diff"
            )
        else:
            key = (
                "CASE2_small_same" if (all_pairs_below_thresh and same_survey) else
                "CASE2_small_diff" if (all_pairs_below_thresh) else
                "CASE2_large_same" if (same_survey) else
                "CASE2_large_diff"
            )

        # Reorder columns for readability
        all_columns = list(group_df.columns)
        ordered_columns = desired_order + [col for col in all_columns if col not in desired_order]
        group_df = group_df.reindex(columns=ordered_columns)

        group_cases[key].append(group_df)

        # -----------------------------------------------
        # Additional tie-breaking investigations / buckets
        # -----------------------------------------------
        flags = set(group_df["z_flag_homogenized"].dropna())
        types = set(group_df["instrument_type_homogenized"].dropna())
        surveys_in_group = set(group_df["survey"].dropna())

        # Case: same flag, different type, different surveys — split by size
        if len(surveys_in_group) > 1 and len(flags) == 1 and len(types) > 1:
            if len(group_df) == 2:
                group_cases["TIE_FLAG_TYPE_BREAK_PAIR"].append(group_df)
            elif len(group_df) > 2:
                group_cases["TIE_FLAG_TYPE_BREAK_GROUP"].append(group_df)

        # Case: same flag, different types — only for true groups (>2)
        if len(flags) == 1 and len(types) > 1 and len(group_df) > 2:
            group_cases["SAME_FLAG_DIFF_TYPE"].append(group_df)

        # NEW: Case — pairs with the same `source` (normalized, non-null, exact match)
        if len(group_df) == 2:
            src_series = group_df["source"]
            src_valid = src_series.dropna().astype(str).str.strip().str.lower()
            if len(src_valid) == 2 and len(set(src_valid)) == 1:
                group_cases["SAME_SOURCE_PAIR"].append(group_df)

        processed_groups += 1

    print(f"✅ Processed {processed_groups} unique groups.")

    # Human-readable descriptions for each bucket
    case_descriptions = {
        "CASE1_small_same": f"pair with delta_z <= {threshold} from same survey",
        "CASE1_small_diff": f"pair with delta_z <= {threshold} from different surveys",
        "CASE1_large_same": f"pair with delta_z > {threshold} from same survey",
        "CASE1_large_diff": f"pair with delta_z > {threshold} from different surveys",
        "CASE2_small_same": f"group with all delta_z <= {threshold} from same survey",
        "CASE2_small_diff": f"group with all delta_z <= {threshold} from different surveys",
        "CASE2_large_same": f"group with some delta_z > {threshold} from same survey",
        "CASE2_large_diff": f"group with some delta_z > {threshold} from different surveys",
        "TIE_FLAG_TYPE_BREAK_PAIR": "pair with equal z_flag_homogenized, different instrument_type_homogenized, and different surveys",
        "TIE_FLAG_TYPE_BREAK_GROUP": "group with equal z_flag_homogenized, different instrument_type_homogenized, and different surveys",
        "SAME_FLAG_DIFF_TYPE": "group with same z_flag_homogenized but at least one differing instrument_type_homogenized",
        "SAME_SOURCE_PAIR": "pair with identical source (normalized, non-null)"  # NEW
    }

    # Final columns to display for examples
    final_columns = [
        "CRD_ID", "ra", "dec", "z", "z_flag", "z_err",
        "z_flag_homogenized", "instrument_type", "instrument_type_homogenized",
        "tie_result", "survey", "source", "compared_to"
    ]

    def survey_signature(df):
        """
        Signature = sorted tuple of the group's surveys (for diversity sampling).
        Example: ('VANDELS', 'VVDS')
        """
        vals = df["survey"].dropna().astype(str).unique().tolist()
        return tuple(sorted(vals)) if len(vals) else ("<MISSING>",)

    MAX_EXAMPLES_PER_CASE = 5  # show up to 5 groups per bucket

    for case_name, groups in group_cases.items():
        if not groups:
            continue

        print(f"\n📌 Showing examples of {case_descriptions[case_name]} "
              f"({len(groups)} groups found):")

        # 1) Prefer groups with unique survey signatures to increase diversity
        seen_sigs = set()
        diverse_selection = []
        leftovers = []

        for g in groups:
            sig = survey_signature(g)
            if sig not in seen_sigs:
                seen_sigs.add(sig)
                diverse_selection.append(g)
            else:
                leftovers.append(g)

            if len(diverse_selection) >= MAX_EXAMPLES_PER_CASE:
                break

        # 2) If needed, fill remaining slots with leftover groups (allow repetition)
        i = 0
        while len(diverse_selection) < MAX_EXAMPLES_PER_CASE and i < len(leftovers):
            diverse_selection.append(leftovers[i])
            i += 1

        # 3) Display in order with the final selected columns
        for group in diverse_selection:
            group_to_show = group.reindex(columns=final_columns)
            display(group_to_show)
            print("-" * 80)
else:
    print("If you want to do the manual comparisson, change the variable in the top of this cell to True. Be aware that this can take a long time.")

## Validation - Prepared Catalogs

In [ ]:
# Dicionário de regras equivalente ao YAML
translation_rules = {
    "2DFGRS": {
        "z_flag_translation": {1: 0, 2: 1, 3: 3, 4: 4, 5: 4},
        "instrument_type_translation": {"default": "s"},
    },
    "2DFLENS": {
        "z_flag_translation": {1: 0, 2: 1, 3: 3, 4: 4, 6: 6},
        "instrument_type_translation": {"default": "s"},
    },
    "2MRS": {
        "z_flag_translation": {
            "conditions": [
                {"expr": "z_err == 0", "value": 3},
                {"expr": "0 < z_err < 0.0005", "value": 4},
                {"expr": "z_err >= 0.0005", "value": 3},
            ],
            "default": 0,
        },
        "instrument_type_translation": {"default": "s"},
    },
    "3D-HST": {
        "z_flag_translation": {
            "conditions": [
                {"expr": "z_best_s == 0", "value": 6},
                {"expr": "z_best_s == 1 and z_spec != -1", "value": 4},
                {"expr": "z_best_s == 2 and use_zgrism == 1 and flag1 == 0 and flag2 == 0", "value": 3},
                {"expr": "z_best_s == 3 and use_phot == 1", "value": 3},
            ],
            "default": 0,
        },
        "instrument_type_translation": {
            "conditions": [
                {"expr": "z_best_s == 1", "value": "s"},
                {"expr": "z_best_s == 2", "value": "g"},
                {"expr": "z_best_s == 3", "value": "p"},
            ],
            "default": "g",
        },
    },
    "6DFGS": {
        "z_flag_translation": {1: 0, 2: 1, 3: 3, 4: 4, 6: 6},
        "instrument_type_translation": {"default": "s"},
    },
    "ASTRODEEP": {
        "z_flag_translation": {
            "conditions": [
                {"expr": "zspec_survey != '-'", "value": 4},
                {"expr": "zspec_survey == '-'", "value": 3},
            ],
            "default": 0,
        },
        "instrument_type_translation": {
            "conditions": [
                {"expr": "zspec_survey != '-'", "value": "s"},
                {"expr": "zspec_survey == '-'", "value": "p"},
            ],
            "default": "p",
        },
    },
    "ASTRODEEP-JWST": {
        "z_flag_translation": {
            "conditions": [
                {"expr": "zspec != -99 and z_flag < 400 and (len(str(int(z_flag))) <= 1 or int(str(int(z_flag))[-2]) <= 3)", "value": 4},
                {"expr": "zspec == -99 and z_flag < 400 and (len(str(int(z_flag))) <= 1 or int(str(int(z_flag))[-2]) <= 3)", "value": 3},
            ],
            "default": 0,
        },
        "instrument_type_translation": {
            "conditions": [
                {"expr": "zspec != -99", "value": "s"},
                {"expr": "zspec == -99", "value": "p"},
            ],
            "default": "p",
        },
    },
    "DESI": {
        "z_flag_translation": {
            "conditions": [
                {"expr": "ZCAT_PRIMARY != True", "value": 0},
                {"expr": "z_flag != 0 and ZCAT_PRIMARY == True", "value": 1},
                {"expr": "z_flag == 0 and ZCAT_PRIMARY == True and z_err < 0.0005", "value": 4},
                {"expr": "z_flag == 0 and ZCAT_PRIMARY == True and z_err >= 0.0005", "value": 3},
            ],
            "default": 0,
        },
        "instrument_type_translation": {"default": "s"},
    },
    "JADES": {
        "z_flag_translation": {4: 4, 3: 3, 2: 2, 1: 1, 0: 0},
        "instrument_type_translation": {"default": "s"},
    },
    "MOSDEF": {
        "z_flag_translation": {7: 4, 6: 3, 5: 2, 4: 2, 3: 1, 2: 1, 1: 0, 0: 0},
        "instrument_type_translation": {"default": "s"},
    },
    "OZDES": {
        "z_flag_translation": {1: 0, 2: 1, 3: 3, 4: 4, 6: 6},
        "instrument_type_translation": {"default": "s"},
    },
    "PRIMUS": {
        "z_flag_translation": {-1: 0, 2: 1, 3: 2, 4: 3},
        "instrument_type_translation": {"default": "g"},
    },
    "VANDELS": {
        "z_flag_translation": {
            0: 0, 1: 1, 2: 2, 3: 4, 4: 4, 9: 3,
            10: 0, 11: 1, 12: 2, 13: 4, 14: 4, 19: 3,
            20: 0, 21: 1, 22: 2, 23: 4, 24: 4, 29: 3,
            210: 0, 211: 1, 212: 2, 213: 4, 214: 4, 219: 3,
        },
        "instrument_type_translation": {"default": "s"},
    },
    "VIMOS": {
        "z_flag_translation": {4: 4, 3: 3, 2: 2, 1: 1, 0: 0},
        "instrument_type_translation": {"default": "s"},
    },
    "VUDS": {
        "z_flag_translation": {
            1: 1, 11: 1, 21: 1, 31: 1, 41: 1,
            2: 2, 12: 2, 22: 2, 32: 2, 42: 2, 9: 2, 19: 2, 29: 2, 39: 2, 49: 2,
            3: 3, 13: 3, 23: 3, 33: 3, 43: 3,
            4: 4, 14: 4, 24: 4, 34: 4, 44: 4,
        },
        "instrument_type_translation": {"default": "s"},
    },
    "VVDS": {
        "z_flag_translation": {
            0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 9: 2,
            10: 0, 11: 1, 12: 2, 13: 3, 14: 4, 19: 2,
            20: 0, 21: 1, 22: 2, 23: 3, 24: 4, 29: 2,
            210: 0, 211: 1, 212: 2, 213: 3, 214: 4, 219: 2,
        },
        "instrument_type_translation": {"default": "s"},
    },

    # Special cases using continuous rule and inherited type
    "CANDELS": {"_special": "CANDELS_NED"},
    "NED": {"_special": "CANDELS_NED"},
}

def _safe_eval_expr(expr: str, ctx: dict) -> bool:
    """
    Avalia 'expr' usando apenas variáveis do ctx e funções básicas.
    Retorna True/False; se der erro, retorna False.
    """
    try:
        # Permitir apenas funções básicas e numpy
        allowed_globals = {
            "__builtins__": {"len": len, "int": int, "str": str, "float": float},
            "np": np,
        }
        return bool(eval(expr, allowed_globals, ctx))
    except Exception:
        return False

def _apply_translation(value_map, row_ctx):
    """
    value_map pode ser:
      - dict simples {orig: dest} (pode conter 'default')
      - dict com 'conditions' (lista de {expr, value}) e opcional 'default'
    Retorna (valor_traduzido, matched_bool)
    """
    if isinstance(value_map, dict) and "conditions" in value_map:
        for cond in value_map["conditions"]:
            expr = cond.get("expr", "")
            val = cond.get("value", np.nan)
            if expr and _safe_eval_expr(expr, row_ctx):
                return val, True
        # nenhum matched -> usa default se houver
        if "default" in value_map:
            return value_map["default"], True
        return np.nan, False

    # mapeamento direto (sem 'conditions'):
    if isinstance(value_map, dict):
        key = row_ctx.get("z_flag", np.nan)
        if key in value_map:
            return value_map[key], True
        # Se não houver chave correspondente, mas existir 'default', use-o
        if "default" in value_map:
            return value_map["default"], True
        return np.nan, False

    return np.nan, False

def validate_row(row):
    survey = row.get("survey", None)

    # construir contexto com None -> np.nan, para evitar erros de comparação
    ctx = {}
    for k, v in row.items():
        ctx[k] = (np.nan if v is None else v)

    # Casos especiais (CANDELS e NED): regra contínua 0..1 e type herdado
    if survey in ("CANDELS", "NED"):
        x = row.get("z_flag", np.nan)
        # z_flag esperado:
        if x == 0.0:
            z_expected = 0.0
        elif (isinstance(x, (float, int))) and (0.0 < x < 0.7):
            z_expected = 1.0
        elif (isinstance(x, (float, int))) and (0.7 <= x < 0.9):
            z_expected = 2.0
        elif (isinstance(x, (float, int))) and (0.9 <= x < 0.99):
            z_expected = 3.0
        elif (isinstance(x, (float, int))) and (0.99 <= x <= 1.0):
            z_expected = 4.0
        else:
            z_expected = np.nan

        # type_expected é o próprio 'type' da linha
        type_expected = row.get("instrument_type", np.nan)
        return z_expected, type_expected

    # Regras gerais dos surveys
    rules = translation_rules.get(survey, None)
    if rules is None:
        return np.nan, np.nan

    # z_flag_homogenized esperado
    z_rules = rules.get("z_flag_translation", None)
    if z_rules is None:
        z_expected = np.nan
    else:
        z_expected, _ = _apply_translation(z_rules, ctx)

    # instrument_type_homogenized esperado
    t_rules = rules.get("instrument_type_translation", None)
    if t_rules is None:
        type_expected = np.nan
    else:
        if isinstance(t_rules, dict) and ("conditions" in t_rules or "default" in t_rules):
            type_expected, matched = _apply_translation(t_rules, ctx)
        else:
            type_expected, matched = _apply_translation(t_rules, ctx)


    return z_expected, type_expected


# =========================================================
# VALIDATE TRANSLATIONS IN TEMP FILES
# =========================================================
merged_files = glob.glob(os.path.join(prepared_temp_dir, "prepared*/*.parquet"))
merged_files = [f for f in merged_files if "pipeline_sample" not in f]

if not merged_files:
    print("⚠️ No prepared parquet files found for validation.")
else:
    issues = []
    
    for merged_file in merged_files:
        print(f"🔍 Validating {merged_file}")
        df = pd.read_parquet(merged_file)
    
        for _, row in df.iterrows():
            z_exp, type_exp = validate_row(row)
    
            if not (pd.isna(z_exp) and pd.isna(row["z_flag_homogenized"])) and z_exp != row["z_flag_homogenized"]:
                issue = row.to_dict()
                issue["field"] = "z_flag_homogenized"
                issue["expected"] = z_exp
                issue["found"] = row["z_flag_homogenized"]
                issues.append(issue)
    
            if not (pd.isna(type_exp) and pd.isna(row["instrument_type_homogenized"])) and type_exp != row["instrument_type_homogenized"]:
                issue = row.to_dict()
                issue["field"] = "instrument_type_homogenized"
                issue["expected"] = type_exp
                issue["found"] = row["instrument_type_homogenized"]
                issues.append(issue)
    
    if issues:
        issues_df = pd.DataFrame(issues)
        display(issues_df)
        print(f"⚠️ {len(issues)} mismatches found!")
    else:
        print("✅ All homogenized fields match the expected values.")

# Time Profiler

In [ ]:
import os
import re
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from collections import defaultdict

# ============================================
# 1. CONFIGURAÇÃO
# ============================================

log_dir = "process001/process_info"

log_files = [
    "prepare_all.log",
    "import_all.log",
    "margin_cache_all.log",
    "crossmatch_and_merge_all.log",
    "process.log"
]

START_RE = re.compile(
    r"(?P<timestamp>\d{4}-\d{2}-\d{2}-\d{2}:\d{2}:\d{2}\.\d+): Starting: (?P<task>[\w_]+) id=(?P<id>[\w\d_]+)"
)
FINISH_RE = re.compile(
    r"(?P<timestamp>\d{4}-\d{2}-\d{2}-\d{2}:\d{2}:\d{2}\.\d+): Finished: (?P<task>[\w_]+) id=(?P<id>[\w\d_]+)"
)

# Vamos manter SEMPRE o primeiro start e o último finish de cada task|id
start_times = {}
end_times = {}

# ============================================
# 2. LEITURA E PARSE DOS LOGS
# ============================================

for file in log_files:
    path = os.path.join(log_dir, file)
    if not os.path.exists(path):
        continue

    with open(path) as f:
        for line in f:
            m_start = START_RE.search(line)
            m_finish = FINISH_RE.search(line)

            if m_start:
                task_id = f"{m_start.group('task')}|{m_start.group('id')}"
                ts = datetime.strptime(m_start.group("timestamp"), "%Y-%m-%d-%H:%M:%S.%f")
                # guarda o menor (primeiro) start
                if task_id not in start_times:
                    start_times[task_id] = ts
                else:
                    if ts < start_times[task_id]:
                        start_times[task_id] = ts

            if m_finish:
                task_id = f"{m_finish.group('task')}|{m_finish.group('id')}"
                ts = datetime.strptime(m_finish.group("timestamp"), "%Y-%m-%d-%H:%M:%S.%f")
                # guarda o maior (último) finish
                if task_id not in end_times:
                    end_times[task_id] = ts
                else:
                    if ts > end_times[task_id]:
                        end_times[task_id] = ts

# ============================================
# 3. CONSTRUÇÃO DO EIXO Y EM ORDEM CUSTOMIZADA
# ============================================

# Considere apenas tasks que têm start e end
all_ids = sorted(set(start_times) & set(end_times))

pipeline_init_id = "pipeline_init|pipeline_init"
consolidate_id = "consolidate|consolidate"

# IDs de prepare_catalog individuais (exclui o agregador prepare_catalogs)
prepare_ids = [
    tid for tid in all_ids
    if tid.startswith("prepare_catalog|") and tid != "prepare_catalogs|prepare_catalogs"
]

# Ordene os prepares pelo start: quem começa antes aparece antes -> y menor -> "mais abaixo" visualmente
prepare_ids_sorted = sorted(prepare_ids, key=lambda tid: start_times[tid])

# Import cat0 inicial, se existir
import_cat0 = [tid for tid in all_ids if tid == "import_catalog|cat0_hats"]

# Demais tasks (exceto pipeline_init, consolidate, prepares e import_cat0)
remaining_ids = [
    tid for tid in all_ids
    if tid not in prepare_ids + import_cat0 + [pipeline_init_id, consolidate_id]
]

# Agrupar por step numérico (catX, merged_stepX, etc.) para ordenar dentro dos steps
step_dict = defaultdict(list)
for tid in remaining_ids:
    match = re.search(r"(?:cat|merged_step)(\d+)", tid)
    if match:
        step = int(match.group(1))
        step_dict[step].append(tid)

ordered_step_ids = []
for step in sorted(step_dict):
    step_tasks = step_dict[step]

    def task_order(tid):
        if tid.startswith("import_catalog|cat"):
            return 0
        elif tid.startswith("generate_margin_cache"):
            return 1
        elif tid.startswith("crossmatch_and_merge"):
            return 2
        elif tid.startswith("import_catalog|merged_step"):
            return 3
        else:
            return 99

    # Dentro do step, mantém uma ordem lógica pelas "fases"
    ordered_step_ids.extend(sorted(step_tasks, key=task_order))

# Monta a ordem final: pipeline_init -> prepares (ordenados pelo start) -> import_cat0 -> steps -> consolidate
ordered_ids = []
if pipeline_init_id in all_ids:
    ordered_ids.append(pipeline_init_id)
ordered_ids.extend(prepare_ids_sorted)
ordered_ids.extend(import_cat0)
ordered_ids.extend(ordered_step_ids)
if consolidate_id in all_ids:
    ordered_ids.append(consolidate_id)

# ============================================
# 4. MONTAGEM DOS DADOS PARA O PLOT
# ============================================

# Tempo adicional a ser subtraído do início do pipeline_init (em segundos)
aditional_pipeline_init_time = 3  # ⏱️ ajuste aqui conforme necessário

# Se existir pipeline_init, ajusta o início para "andar" um pouco antes (apenas para estética)
if pipeline_init_id in start_times:
    start_times[pipeline_init_id] -= timedelta(seconds=aditional_pipeline_init_time)

# Zera o tempo no primeiro start dentre os que vamos plotar
start_zero = min(start_times[tid] for tid in ordered_ids)

# --------------------------------------------------
# 🛠️ INSERIR REGISTRO MANUAL DA TAREFA "register"
# --------------------------------------------------
register_id = "register|register"
register_duration = 3  # ⏱️ duração da tarefa "register" em segundos

# Insere o "register" ao final
ordered_ids.append(register_id)
# Começa após o consolidate (se existir) ou após o último término conhecido
if consolidate_id in end_times:
    register_start = max(end_times[consolidate_id], *end_times.values())
else:
    register_start = max(end_times.values())
register_end = register_start + timedelta(seconds=register_duration)

start_times[register_id] = register_start
end_times[register_id] = register_end
# --------------------------------------------------

# Constrói listas relativas ao start_zero
y_labels = []
start_list = []
end_list = []

for tid in ordered_ids:
    y_labels.append(tid)
    start_rel = (start_times[tid] - start_zero).total_seconds()
    end_rel = (end_times[tid] - start_zero).total_seconds()
    start_list.append(start_rel)
    end_list.append(end_rel)

# ============================================
# 4b. AJUSTAR POSIÇÕES Y PARA SEPARAR "register"
# ============================================

# Cria posições Y padrão e separa o último (register) com um espaçamento extra
y_positions = list(range(len(ordered_ids)))
y_positions[-1] += 5.0  # 🛠️ Aumenta a posição do "register" no eixo Y

# ============================================
# 5. PLOTAGEM DO GRÁFICO DE TIME PROFILE
# ============================================

plt.figure(figsize=(12, 4))

# === Mapear cores por grupo
group_colors = {
    "pipeline_init": "#003f5c",       # azul escuro
    "prepare_catalogs": "#b8860b",    # amarelo escuro
    "crossmatch": "#2f855a",          # verde escuro
    "consolidate": "#003f5c",         # mesmo do pipeline_init
    "register": "#003f5c",            # mesmo azul escuro do pipeline_init
}

# === Determinar grupo de cada tarefa
def get_group(tid):
    if tid == pipeline_init_id:
        return "pipeline_init"
    elif tid == register_id:
        return "register"
    elif tid in prepare_ids:
        return "prepare_catalogs"
    elif tid == consolidate_id:
        return "consolidate"
    else:
        return "crossmatch"

# === Plotar tarefas com cor unificada para linha e bolinhas
for y, start, end, tid in zip(y_positions, start_list, end_list, ordered_ids):
    group = get_group(tid)
    color = group_colors[group]
    plt.hlines(y, start, end, colors=color, linewidth=2)
    plt.scatter(start, y, color=color, s=10)  # início
    plt.scatter(end, y, color=color, s=10)    # fim

# ============================================
# Agrupar labels do eixo Y por grupo
# ============================================

group_positions = defaultdict(list)
for y, tid in zip(y_positions, ordered_ids):
    group_positions[get_group(tid)].append(y)

group_labels = []
group_ticks = []

for label in ["pipeline_init", "prepare_catalogs", "crossmatch", "consolidate", "register"]:
    if group_positions[label]:
        center = sum(group_positions[label]) / len(group_positions[label])
        group_labels.append(label)
        group_ticks.append(center)

# ============================================
# Personalização final do gráfico
# ============================================

plt.yticks(group_ticks, group_labels, fontsize=20)
plt.xticks(fontsize=12)
plt.xlabel("Time (s)", fontsize=20)
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()